Author: Sungguk Cha

eMail : navinad@naver.com

This notebook is an experiment of XOR solvable activation function.

In [1]:
import numpy as np
import torch
import torch.nn
from dataloaders import make_data_loader
from tqdm import tqdm
from utils.perceptron import Perceptron

In [2]:
class Trainer(object):
    def __init__(self, args):
        self.train_loader, self.val_loader, self.test_loader, self.nclass = make_data_loader(args)
        
        self.model = Perceptron(mode=args.mode)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = args.lr)
        
        self.model = torch.nn.DataParallel(self.model, device_ids=args.gpu_ids)
        self.model = self.model.cuda()
        
        if not args.cuda:
            raise NotImplementedError
            
    def train(self, epoch):
        train_loss = 0.0
        self.model.train()
        tbar = tqdm(self.train_loader)
        for i, sample in enumerate(tbar):
            image, target = sample['image'].cuda(), sample['label'].cuda().long()
            self.optimizer.zero_grad()
            output = self.model(image)
            loss = (output - target).pow(2).sum()
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item()
            tbar.set_description('Train loss: %.3f' % (train_loss / (i+1)))
            
    def val(self, epoch):
        test_loss = 0.0
        self.model.eval()
        tbar = tqdm(self.val_loader)
        results = []
        for i, sample in enumerate(tbar):
            image, target = sample['image'].cuda(), sample['label'].cuda()
            with torch.no_grad():
                output = self.model(image)
            loss = (output - target).pow(2).sum()
            input = image.cpu().numpy().tolist()
            output = output.cpu().numpy().tolist()
            results.append ( (input, output) )
            test_loss += loss.item()
            tbar.set_description('Test loss: %.3f' % (test_loss / (i + 1)))
        for item in results:
            print( item )

Experiment 1.

![Results](./images/usual.png)

In [3]:
class Args(object):
    def __init__(self):
        self.dataset = 'xor'
        self.batch_size = 1
        self.lr = 0.01
        self.mode = 'relu'
        self.cuda = True
        self.gpu_ids = [0]

args = Args()
        
epoch = 10
trainer = Trainer(args)
for e in range(0, epoch):
    trainer.train(e)
    trainer.val(e)

ReLU activation function


Test loss: 0.361: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 501.22it/s]


([[0.0, 0.0]], [[0.0]])
([[0.0, 1.0]], [[0.33272916078567505]])
([[1.0, 0.0]], [[0.27797240018844604]])
([[1.0, 1.0]], [[0.6920680999755859]])


Test loss: 0.266: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 401.08it/s]


([[0.0, 0.0]], [[0.29373225569725037]])
([[0.0, 1.0]], [[0.42598024010658264]])
([[1.0, 0.0]], [[0.3748631775379181]])
([[1.0, 1.0]], [[0.5071111917495728]])


...

([[0.0, 0.0]], [[0.4992128312587738]])
([[0.0, 1.0]], [[0.4443220794200897]])
([[1.0, 0.0]], [[0.5447314977645874]])
([[1.0, 1.0]], [[0.4898407459259033]])


Test loss: 0.253: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 572.95it/s]


([[0.0, 0.0]], [[0.48164576292037964]])
([[0.0, 1.0]], [[0.42297717928886414]])
([[1.0, 0.0]], [[0.4803203344345093]])
([[1.0, 1.0]], [[0.4216517508029938]])


Test loss: 0.253: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 446.16it/s]


([[0.0, 0.0]], [[0.5178813338279724]])
([[0.0, 1.0]], [[0.5597406625747681]])
([[1.0, 0.0]], [[0.5420119762420654]])
([[1.0, 1.0]], [[0.5838713049888611]])


Experiment 2.

![Results](./images/xor.png)

In [4]:
class Args(object):
    def __init__(self):
        self.dataset = 'xor'
        self.batch_size = 1
        self.lr = 0.01
        self.mode = 'xor'
        self.cuda = True
        self.gpu_ids = [0]

args = Args()
        
epoch = 10
trainer = Trainer(args)
for e in range(0, epoch):
    trainer.train(e)
    trainer.val(e)

XOR activation function


Test loss: 0.000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 401.08it/s]


([[0.0, 0.0]], [[0.0]])
([[0.0, 1.0]], [[0.9999999403953552]])
([[1.0, 0.0]], [[1.0000001192092896]])
([[1.0, 1.0]], [[0.0]])


Test loss: 0.000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 401.08it/s]


([[0.0, 0.0]], [[0.0]])
([[0.0, 1.0]], [[0.9999999403953552]])
([[1.0, 0.0]], [[1.0000001192092896]])
([[1.0, 1.0]], [[0.0]])


...

([[0.0, 0.0]], [[0.0]])
([[0.0, 1.0]], [[1.0]])
([[1.0, 0.0]], [[1.0]])
([[1.0, 1.0]], [[0.0]])


Test loss: 0.000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 364.63it/s]


([[0.0, 0.0]], [[0.0]])
([[0.0, 1.0]], [[1.0]])
([[1.0, 0.0]], [[1.0]])
([[1.0, 1.0]], [[0.0]])


Test loss: 0.000: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 401.09it/s]


([[0.0, 0.0]], [[0.0]])
([[0.0, 1.0]], [[1.0]])
([[1.0, 0.0]], [[1.0]])
([[1.0, 1.0]], [[0.0]])
